# SA language detection hackathon
https://www.kaggle.com/c/language-identification/overview

### Imports 

In [ ]:
pip install langdetect

     |████████████████████████████████| 983kB 3.3MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=eb73dcbee4ead651065369c58fe72192bc0d7156a849dfe0e3bed8ce4b37f7cb
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
pip install fasttext

     |████████████████████████████████| 71kB 2.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3038437 sha256=d5e0aa98679d625bcec70fabb04f902db405cd9699793622e1830a9ca3ac94cd
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
pip install spacy

In [ ]:
pip install imblearn

In [ ]:
pip install scikit-plot

In [ ]:
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from matplotlib.colors import ListedColormap
%matplotlib inline

# imports for Natural Language  Processing
import re
import os
import nltk
import string
import time
import fasttext
import spacy.cli
from langdetect import detect
import unicodedata
import numpy as np
import pandas as pd
import xgboost
from sklearn import metrics
from nltk.corpus import stopwords
from html.parser import HTMLParser
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
nltk.download('stopwords')
nltk.download('punkt')

# Classification Models

from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier

# Performance Evaluation
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from scikitplot.metrics import plot_roc, plot_confusion_matrix
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix,  make_scorer, fbeta_score

# Import library for train test split
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV


# Ignore warnings
import warnings
warnings.simplefilter(action='ignore')

#spacy
spacy.cli.download('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


### Import data 

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Classification/Hackathon/train_set.csv')

test = pd.read_csv('/content/drive/MyDrive/Classification/Hackathon/test_set.csv')

In [ ]:
train.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [ ]:
test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


## 1. Class balance 

In [ ]:
train['lang_id'].value_counts()

nbl    3000
nso    3000
ssw    3000
eng    3000
ven    3000
zul    3000
xho    3000
afr    3000
sot    3000
tso    3000
tsn    3000
Name: lang_id, dtype: int64

## 2. Explore data 

In [ ]:
def print_plot(index):
    example = train[train.index == index][['text', 'lang_id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('lang_id:', example[1])
print_plot(10)
print_plot(50)
print_plot(80)
print_plot(100)
print_plot(500)

lapho inarha yangeqadi ingenwe ngokungasimthetho begodu kunobujamo obukhethekileko umnikazi wenarha angaya kumasipala wendawo namkha nakutlhogekako emnyangweni wezezindlu ukufna irhelebho nayikuthi unobangela kufuna izindlu
lang_id: nbl
prof frikkie van niekerk mokaedi wa setheo sa merero ya badiri baithuti boitshimololedi le dipatlisiso o ne a bolelela herce-vigil ka ga hisetori ya yunibesithi gammogo le photefolio ya yona ya ditshwanelo tsa botlhalefi
lang_id: tsn
ho na le mananeo a thupello naheng ka bophara bakeng sa batsamaisi ba bommasepala bahlanka ba mesebetsi e ikgethang ditho tsa lekgotla le diprofeshenale tsa meralo tse kenyeletsang le ba bommasepala ba bang
lang_id: sot
u anganelana na sisiteme dza nivho sa tsumbo ndivho ya ngwaniwapo i shumiswa yo angana na thekhinolodzhi ya tshizwino-zwino ya bayothekhinolodzhi kha zwa mishonga na dziwe sekhitha kha u engedzedza tshiimo tsha vhubveledzi
lang_id: ven


## 3. Text preprocessing

In [ ]:
df = train.copy()
df1 = test.copy()

In [ ]:
duplicate = df[df.duplicated('text')] 

In [ ]:
duplicate

,lang_id,text
940,xho,xa umntu lowo ephuma kule ndawo yihlola-hlolen...
1208,nbl,ngokwesekhtjheni yomthetho ophathelene nalokhu...
1252,ssw,kwabelana ngemininingwane enhlanganweni kutakw...
1304,xho,umntu ocela ukukhuselwa ngumbuso ngumntu obale...
1460,ven,kha vha ḓivhe hezwi a vha nga ḓo ita ndingo ya...
...,...,...
32980,ssw,inhloso ye-wua kutsi yente bantfu bendzawo let...
32983,ssw,timiso tesigatjana titawusebenta ngetingucuko ...
32985,nso,ge o nyaka go kgopela phihlelelo ya direkoto t...
32989,ssw,imenenja yesigodzi utakwatisa ngembhalo uma le...


In [ ]:
df3 = df.drop_duplicates(inplace=True)

In [ ]:
df3.shape 

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...
...,...,...
32994,eng,manuel marin s ill-fated debt sources but very...
32995,tsn,popo ya dipolateforomo tse ke go tlisa boetele...
32997,eng,closing date for the submission of completed t...
32998,xho,nawuphina umntu ofunyenwe enetyala phantsi kwa...


In [ ]:
df3['lang_id'].value_counts()

eng    2998
zul    2924
nso    2873
tsn    2869
sot    2833
tso    2758
xho    2659
afr    2641
ven    2605
ssw    2426
nbl    2362
Name: lang_id, dtype: int64

## 4. Train test- validation split 

In [ ]:
df.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [81]:
X = df.text
y = df.lang_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

## 5. Train Model 
Will be testing 6 Models 


### K-nearest-neighbours (KNN)

In [ ]:
knn = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', KNeighborsClassifier()),
              ])
knn.fit(X_train, y_train)

#%%time

y_pred = knn.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,output_dict=True))

accuracy 0.9645454545454546
{'afr': {'precision': 0.991130820399113, 'recall': 0.9988826815642458, 'f1-score': 0.994991652754591, 'support': 895}, 'eng': {'precision': 0.9762931034482759, 'recall': 0.9966996699669967, 'f1-score': 0.9863908546543276, 'support': 909}, 'nbl': {'precision': 0.8849372384937239, 'recall': 0.962457337883959, 'f1-score': 0.9220708446866486, 'support': 879}, 'nso': {'precision': 0.948559670781893, 'recall': 0.9798087141339001, 'f1-score': 0.9639309984317825, 'support': 941}, 'sot': {'precision': 0.9846994535519126, 'recall': 0.9814814814814815, 'f1-score': 0.983087834151664, 'support': 918}, 'ssw': {'precision': 0.9755555555555555, 'recall': 0.9669603524229075, 'f1-score': 0.9712389380530974, 'support': 908}, 'tsn': {'precision': 0.9618497109826589, 'recall': 0.9433106575963719, 'f1-score': 0.9524899828277047, 'support': 882}, 'tso': {'precision': 0.9771689497716894, 'recall': 0.9988331388564761, 'f1-score': 0.9878822850548182, 'support': 857}, 'ven': {'precisi

In [ ]:
report = classification_report(y_test, y_pred,output_dict=True)
results = pd.DataFrame(report).transpose()
results

,precision,recall,f1-score,support
afr,0.991131,0.998883,0.994992,895.000000
eng,0.976293,0.996700,0.986391,909.000000
nbl,0.884937,0.962457,0.922071,879.000000
nso,0.948560,0.979809,0.963931,941.000000
sot,0.984699,0.981481,0.983088,918.000000
ssw,0.975556,0.966960,0.971239,908.000000
tsn,0.961850,0.943311,0.952490,882.000000
tso,0.977169,0.998833,0.987882,857.000000
ven,0.996798,0.997863,0.997330,936.000000
xho,0.963595,0.918655,0.940589,922.000000


### Random Forest

In [ ]:
rf = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', RandomForestClassifier()),
              ])
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
report = classification_report(y_test, y_pred,output_dict=True)
results = pd.DataFrame(report).transpose()
results

accuracy 0.9828282828282828


,precision,recall,f1-score,support
afr,0.996652,0.997765,0.997208,895.000000
eng,0.984832,1.000000,0.992358,909.000000
nbl,0.975294,0.943117,0.958936,879.000000
nso,0.998934,0.995749,0.997339,941.000000
sot,0.995657,0.998911,0.997281,918.000000
ssw,0.987514,0.958150,0.972610,908.000000
tsn,0.996599,0.996599,0.996599,882.000000
tso,0.998834,1.000000,0.999417,857.000000
ven,1.000000,1.000000,1.000000,936.000000
xho,0.976693,0.954447,0.965442,922.000000


### Neural networks

In [ ]:
from sklearn.neural_network import MLPClassifier

nn = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', MLPClassifier(hidden_layer_sizes = [100]*5)),
              ])
nn.fit(X_train, y_train)

y_pred = nn.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
report = classification_report(y_test, y_pred,output_dict=True)
results = pd.DataFrame(report).transpose()
results

accuracy 0.9981818181818182


,precision,recall,f1-score,support
afr,0.998883,0.998883,0.998883,895.000000
eng,0.997805,1.000000,0.998901,909.000000
nbl,0.995455,0.996587,0.996020,879.000000
nso,1.000000,0.994687,0.997336,941.000000
sot,0.995662,1.000000,0.997826,918.000000
ssw,1.000000,0.998899,0.999449,908.000000
tsn,0.997732,0.997732,0.997732,882.000000
tso,1.000000,1.000000,1.000000,857.000000
ven,1.000000,1.000000,1.000000,936.000000
xho,0.996750,0.997831,0.997290,922.000000


### Naive Bayes

In [ ]:
nb = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
report = classification_report(y_test, y_pred,output_dict=True)
results = pd.DataFrame(report).transpose()
results

accuracy 0.9985858585858586


,precision,recall,f1-score,support
afr,0.997770,1.000000,0.998884,895.000000
eng,0.996711,1.000000,0.998353,909.000000
nbl,0.993213,0.998862,0.996029,879.000000
nso,1.000000,0.998937,0.999468,941.000000
sot,0.998912,1.000000,0.999456,918.000000
ssw,1.000000,1.000000,1.000000,908.000000
tsn,1.000000,0.998866,0.999433,882.000000
tso,1.000000,1.000000,1.000000,857.000000
ven,1.000000,1.000000,1.000000,936.000000
xho,0.998914,0.997831,0.998372,922.000000


### Linear SVM

In [ ]:
linsvc = Pipeline([('tfidf', TfidfVectorizer()),
                   ('clf', LinearSVC())])

linsvc.fit(X_train, y_train)

y_pred = linsvc.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
report = classification_report(y_test, y_pred,output_dict=True)
results = pd.DataFrame(report).transpose()
results

accuracy 0.9973737373737374


,precision,recall,f1-score,support
afr,0.998883,0.998883,0.998883,895.000000
eng,0.997805,1.000000,0.998901,909.000000
nbl,0.995439,0.993174,0.994305,879.000000
nso,1.000000,0.996812,0.998403,941.000000
sot,0.996743,1.000000,0.998369,918.000000
ssw,1.000000,0.997797,0.998897,908.000000
tsn,0.997732,0.997732,0.997732,882.000000
tso,1.000000,1.000000,1.000000,857.000000
ven,1.000000,1.000000,1.000000,936.000000
xho,0.991379,0.997831,0.994595,922.000000


### SGD Classifier 



In [ ]:
sgd = Pipeline([('tfidf', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
report = classification_report(y_test, y_pred,output_dict=True)
results = pd.DataFrame(report).transpose()
results

accuracy 0.9888888888888889


,precision,recall,f1-score,support
afr,0.996656,0.998883,0.997768,895.000000
eng,0.990196,1.000000,0.995074,909.000000
nbl,0.990783,0.978385,0.984545,879.000000
nso,0.996785,0.988310,0.992529,941.000000
sot,0.995662,1.000000,0.997826,918.000000
ssw,0.997750,0.976872,0.987201,908.000000
tsn,0.989853,0.995465,0.992651,882.000000
tso,0.986191,1.000000,0.993048,857.000000
ven,0.996805,1.000000,0.998400,936.000000
xho,0.958991,0.989154,0.973839,922.000000


Sent 6 models to predict test data, results are as follow (F1-Score) - second results are after doing preprocessing: 

1.   NB: 0.95533      || 0.95423
2.   LinSCV: 0.92563  || 0.93435
3.   SGD: 0.92365  || 0.92387
4.   KNN: 0.88536 || 0.88514
5.   RF: 0.86204 
6.   NN: 0.85610 



## 6. Hypertunning 

### NB

In [ ]:
MultinomialNB().get_params().keys()

dict_keys(['alpha', 'class_prior', 'fit_prior'])

In [88]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

In [90]:
from sklearn.metrics import classification_report
ftwo_scorer = make_scorer(fbeta_score, beta=2, average='micro')
clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=ftwo_scorer)

clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test), digits=4))

KeyboardInterrupt: ignored

In [ ]:
print(clf.best_score_)
print(clf.best_estimator_)

0.9992640692640693
Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l1', smooth_idf=True,
                                  sublinear_tf=False, use_idf=False)),
                ('clf',
                 MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))],
         verbose=False)


In [ ]:
test_X = df1.text
y_pred = clf.predict(test_X)
df1['lang_id'] = y_pred

In [ ]:
df1[['index','lang_id']].to_csv('testsubmission_nb1.csv', index=False)

F1 Score on test dataset before one vs rest increased to **0.96259**


## 7. Ensemble of Binary Classifiers (One-vs-Rest)

### NB

### Testing without resampling after removing duplicates 


In [ ]:
X = df3.text
y = df3.lang_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
nb1 = Pipeline([('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l1', smooth_idf=True,
                                  sublinear_tf=False, use_idf=False)),
                ('clf', OneVsRestClassifier(MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))),
              ])
nb1.fit(X_train, y_train)

y_pred = nb1.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
report = classification_report(y_test, y_pred,output_dict=True)
results = pd.DataFrame(report).transpose()
results

accuracy 0.9987757373400111


,precision,recall,f1-score,support
afr,1.000000,1.000000,1.000000,806.000000
eng,0.993492,1.000000,0.996736,916.000000
nbl,1.000000,0.994236,0.997110,694.000000
nso,1.000000,0.998818,0.999409,846.000000
sot,0.998843,1.000000,0.999421,863.000000
ssw,0.998645,1.000000,0.999322,737.000000
tsn,1.000000,1.000000,1.000000,844.000000
tso,1.000000,1.000000,1.000000,807.000000
ven,1.000000,1.000000,1.000000,765.000000
xho,1.000000,0.997487,0.998742,796.000000


In [ ]:
test_X = df1.text
y_pred = clf.predict(test_X)
df1['lang_id'] = y_pred

In [ ]:
df1[['index','lang_id']].to_csv('testsubmission_nb1.csv', index=False)

The score reduced after I removed duplicates from 0.999090.. to **0.9987757373400111** There I did not use this model on the test dataset


### Testing with resampling after duplicate removal 


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=2)

In [ ]:
countvect = CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)
X_vector = countvect.fit_transform(X)

tfidf = TfidfTransformer(norm='l1', smooth_idf=True,
                                  sublinear_tf=False, use_idf=False)

X_vector = tfidf.fit_transform(X_vector)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vector, y, test_size=0.3, random_state=42)

In [ ]:
X_train_smote, y_train_smote = smote.fit_sample(X_train, y_train.ravel())

In [ ]:
from collections import Counter 

In [ ]:
print('Before SMOTE:', Counter(y_train))
print('After SMOTE:', Counter(y_train_smote))

Before SMOTE: Counter({'eng': 2082, 'nso': 2027, 'tsn': 2025, 'zul': 2013, 'sot': 1970, 'tso': 1951, 'xho': 1863, 'ven': 1840, 'afr': 1835, 'ssw': 1689, 'nbl': 1668})
After SMOTE: Counter({'sot': 2082, 'tso': 2082, 'nso': 2082, 'tsn': 2082, 'eng': 2082, 'xho': 2082, 'afr': 2082, 'ven': 2082, 'ssw': 2082, 'zul': 2082, 'nbl': 2082})


In [78]:
X_train_smote.shape

(22902, 141958)

In [80]:
y_train_smote.shape

(22902,)

In [79]:
y_test.shape

(8985,)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
dnns_classifier = OneVsRestClassifier(MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))
dnns_classifier.fit(X_train_smote, y_train_smote.ravel())
dnns_predictions_labels = dnns_classifier.predict(X_train_smote)

In [ ]:
#print('accuracy %s' % accuracy_score(dnns_predictions_labels, y_test))
report = classification_report(y_test, dnns_predictions_labels)
results = pd.DataFrame(report).transpose()
results

ValueError: ignored

I attempted to rebalance the data again once I removed the duplicates, however I wasnt able to complete it as I got Value Error. The y_train_smote and the y_test have different shapes. My final score on the board **0.96501** which was achieved through the orginal balanced data with modification hyperparamter tunning and one-vs-rest. 

